In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#!pip install segmentation_models

import os
import cv2
import tensorflow as tf
import keras 
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import os
from PIL import Image
from collections import defaultdict
import gc
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import  Adam
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.callbacks import ModelCheckpoint,Callback
#import segmentation_models as sm
#model = sm.Unet('resnet34', encoder_weights='imagenet')
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

from keras.models import Model, load_model
from keras.layers import AveragePooling2D,MaxPooling2D,Dense,Dropout,Flatten

from keras.models import load_model
%matplotlib inline

# Any results you write to the current directory are saved as output.

In [ ]:
from albumentations import (
    PadIfNeeded,
    HorizontalFlip,
    VerticalFlip,    
    CenterCrop,    
    Crop,
    Compose,
    Transpose,
    RandomRotate90,
    ElasticTransform,
    GridDistortion, 
    OpticalDistortion,
    RandomSizedCrop,
    OneOf,
    CLAHE,
    RandomBrightnessContrast,    
    RandomGamma,
    RandomContrast,
    InvertImg,
    RandomContrast
)

import segmentation_models as sm

BACKBONE = 'resnet34'
preprocess_input = sm.backbones.get_preprocessing(BACKBONE)
model = sm.Unet(BACKBONE,input_shape=(256, 1600,3),classes=4, activation='sigmoid',encoder_weights='imagenet')

model.save("seg_model.h5")

In [ ]:
os.listdir('/kaggle/input/severstal-steel-defect-detection')
train_df = pd.read_csv('/kaggle/input/severstal-steel-defect-detection/train.csv')
train_df['Image_id']= train_df[train_df.columns[0]].apply(lambda x : x.split('_')[0])
train_df['class_id']= train_df[train_df.columns[0]].apply(lambda x : x.split('_')[1])
train_df['mask']= ~train_df[train_df.columns[1]].isna()
print(train_df.shape)
train_df.head(5)

In [ ]:
mask_count=train_df.groupby(by='Image_id').agg(np.sum).reset_index()
print(mask_count.shape)
mask_count.head(3)

In [ ]:
mask_count.sort_values(by='mask',ascending=False,inplace=True)
mask_count.head()

In [ ]:
sub_df = pd.read_csv('/kaggle/input/severstal-steel-defect-detection/sample_submission.csv')
sub_df['Image_id']=sub_df[sub_df.columns[0]].apply(lambda x : x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['Image_id'].unique(),columns=['Image_id'])
print(test_imgs.shape)
test_imgs.head()

In [ ]:
train_path = '/kaggle/input/severstal-steel-defect-detection/train_images/'
train_dir=os.listdir(train_path)
train_dir[:3]
test_path = '/kaggle/input/severstal-steel-defect-detection/test_images/' 
test_dir = os.listdir(test_path)

In [ ]:
def mask2rle(img):
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros((*input_shape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, input_shape)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles


In [ ]:
#model_class = load_model('/kaggle/input/classification-model/class_model.h5')
#model_class.load_weights('/kaggle/input/classifier-model-weight/classifier.h5')
import sys
import os
import zipfile
pa='/kaggle/input/efficientnet'
    
    
sys.path.append(os.path.abspath('/kaggle/input/efficientnet-models/efficientnet-master/'))
from efficientnet.keras import EfficientNetB2,EfficientNetB3
N_CLASSES=4
def ClsModel(n_classes=1, input_shape=(254,600,3)):
    base_model = EfficientNetB2(weights=None, include_top=False, input_shape=input_shape)
    x = AveragePooling2D(pool_size=(5,5), name='avg_pool')(base_model.output)
    x = Flatten()(x)
    x = Dense(1600, activation='relu', name='dense_post_pool')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_classes, activation='sigmoid', name='predictions')(x)
    model = Model(inputs=base_model.input, output=output)
    return model
model = ClsModel(n_classes=N_CLASSES)
N_CLASSES=4
def ClsModel(n_classes=1, input_shape=(254,254,3)):
    base_model = EfficientNetB3(weights=None, include_top=False, input_shape=input_shape)
    x = AveragePooling2D(pool_size=(3,3), name='avg_pool')(base_model.output)
    x = Flatten()(x)
    x = Dense(1024, activation='relu', name='dense_post_pool')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_classes, activation='sigmoid', name='predictions')(x)
    model = Model(inputs=base_model.input, output=output)
    return model
model_class = ClsModel(n_classes=N_CLASSES)
#model_class.load_weights('/kaggle/input/efficient977/EfficientNetB3_efficenent_exp_2.h5')

In [ ]:
# batch size is fixed and 1 can include  patch size
class DataGenerator1(keras.utils.Sequence):
    'Generates data for Keras'
    
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path=train_path,val ='no',augmentaion =[1],
                 batch_size=1, dim=(254, 600), n_channels=3,
                 n_classes=4, random_state=1994, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.val=val
        self.augmentaion=augmentaion
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        
        indexes = self.indexes[self.batch_size*index:self.batch_size*(index+1)]

        # Find list of IDs

        ID = [self.list_IDs[k] for k in indexes]
        X = self.__generate_X(ID)
        
        if self.mode == 'fit':
            y = self.__generate_y(ID)   
            return X,y # preprocess as resnet34  
        elif self.mode == 'predict':
            return X
        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, ID):
        'Generates data containing batch_size samples'
        # Initialization 
        img_i = np.empty((self.batch_size,*self.dim,self.n_channels),dtype=np.uint8)
        

        # Generate data
        for i,loc in enumerate(ID):
            im_name = self.df['Image_id'].loc[loc]
            img_path = f"{self.base_path}/{im_name}"
            img=cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img=cv2.resize(img ,(600,254))
            #img_i[i,] = img 
            
            if self.augmentaion[0]==1 : # don't change
                img_i[i,] = img
                continue;
            aug_no =0
            aug = self.augmentaion[aug_no](p=1)
            img = aug(image=img)['image']
            img_i[i,] = img
        #rgb
        return img_i

    
    def __generate_y(self, ID):
        # image name
        y = np.empty((self.batch_size,self.n_classes))
        for i,loc in enumerate(ID):
            y[i,] = self.df['mask'].loc[loc]
        return y
    
    
    def __load_rgb(self, img_path):
        return cv2.imread(img_path)
 

In [ ]:
# batch size is fixed and 1 can include  patch size
class DataGenerator2(keras.utils.Sequence):
    'Generates data for Keras'
    
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path=train_path,val ='no',augmentaion =[1],
                 batch_size=1, dim=(254, 254), n_channels=3,
                 n_classes=4, random_state=1994, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.val=val
        self.augmentaion=augmentaion
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        
        indexes = self.indexes[self.batch_size*index:self.batch_size*(index+1)]

        # Find list of IDs

        ID = [self.list_IDs[k] for k in indexes]
        X = self.__generate_X(ID)
        
        if self.mode == 'fit':
            y = self.__generate_y(ID)   
            return X,y # preprocess as resnet34  
        elif self.mode == 'predict':
            return X
        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, ID):
        'Generates data containing batch_size samples'
        # Initialization 
        img_i = np.empty((self.batch_size,*self.dim,self.n_channels),dtype=np.uint8)
        

        # Generate data
        for i,loc in enumerate(ID):
            im_name = self.df['Image_id'].loc[loc]
            img_path = f"{self.base_path}/{im_name}"
            img=cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img=cv2.resize(img ,(254,254))
            #img_i[i,] = img 
            
            if self.augmentaion[0]==1 : # don't change
                img_i[i,] = img
                continue;
            aug_no =0
            aug = self.augmentaion[aug_no](p=1)
            img = aug(image=img)['image']
            img_i[i,] = img
        #rgb
        return img_i

    
    def __generate_y(self, ID):
        # image name
        y = np.empty((self.batch_size,self.n_classes))
        for i,loc in enumerate(ID):
            y[i,] = self.df['mask'].loc[loc]
        return y
    
    
    def __load_rgb(self, img_path):
        return cv2.imread(img_path)
 

In [ ]:
model_class_2 = load_model('/kaggle/input/efficent-model-and-weights-975/efficent_class_model.h5')


In [ ]:
all_classifer_path=['/kaggle/input/efficent-b2-600/EfficientNetB3_efficenent_254_600_exp_4.h5',
    '/kaggle/input/efficient977/EfficientNetB3_efficenent_exp_2.h5']

#model_class_2 = load_model('/kaggle/input/efficent-model-and-weights-975/efficent_class_model.h5')

class_models =[model,model_class]

pred_class_i=[]
pred=[]


DataGenerator=[DataGenerator1,DataGenerator2]
for i in range(2):
    augmentaion=[1,HorizontalFlip,VerticalFlip]
    for aug in augmentaion:  
        class_models[i].load_weights(all_classifer_path[i])

        test_generator = DataGenerator[i](
                test_imgs.index,
                df=test_imgs,
                shuffle=False,
                mode='predict',
                base_path='../input/severstal-steel-defect-detection/test_images',
                target_df=test_imgs,
                augmentaion=[aug],
                batch_size=1,
                n_classes=4
            )

        batch_pred_class = class_models[i].predict_generator(
                test_generator, 
                workers=1,
                verbose=1,
                use_multiprocessing=False
            )
        pred.append(batch_pred_class.round().astype(int))
    
    

np.sum(((pred_class_i[0]+pred_class_i[1]+pred_class_i[2])/3).round().astype(int),axis=0)

pred=pred_class_i[0].round()*1.3+pred_class_i[1].round()+pred_class_i[2].round()

pred[pred==1]=0
pred[pred>=2]=1
pred=pred.round().astype('int')
np.sum(pred,axis=0)

In [ ]:
test=pred.copy()

In [ ]:
np.array(test).shape

In [ ]:
test_1=np.array(test)
test_1=test_1[0]+test_1[1]+test_1[2]+test_1[3]+test_1[4]+test_1[5]
test_1[np.where(test_1<=2)]=0
test_1[np.where(test_1>2)]=1
np.sum(test_1,axis=0)

In [ ]:
test_1

test=pred_class_i[0].round().astype(int).copy()
np.sum(test.round().astype(int),axis=0)

In [ ]:
np.sum(test_1.round().astype(int),axis=0)

In [ ]:
aaa=[128/43/741/120]

In [ ]:
np.sum(batch_pred_class)

In [ ]:
pred_class =test_1.round().astype(int)

In [ ]:
np.sum(pred_class,axis=0)

In [ ]:
palet = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]

def mask_image(img,mask):
    img_i = img

    mask_i=mask
    for ch in range(4):
        contours, _ = cv2.findContours(mask_i[:, :, ch], cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)        
        for i in range(0, len(contours)):
            cv2.polylines(img_i, contours[i], True, palet[ch], 2)
        
    return img_i

In [ ]:

# batch size is fixed and 1 can include  patch size
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path=train_path,
                 batch_size=1, dim=(256, 1600), n_channels=3,
                 n_classes=4, random_state=1994, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    
    def __getitem__(self, index):
            'Generate one batch of data'
            # Generate indexes of the batch
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

            # Find list of IDs
            list_IDs_batch = [self.list_IDs[k] for k in indexes]

            X = self.__generate_X(list_IDs_batch)

            if self.mode == 'fit':
                y = self.__generate_y(list_IDs_batch)
                return X, y

            elif self.mode == 'predict':
                return X

            else:
                raise AttributeError('The mode parameter should be set to "fit" or "predict".')
            
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, ID):
        'Generates data containing batch_size samples'
        # Initialization        
        # Generate data
        img_i = np.empty((self.batch_size,*self.dim,self.n_channels),dtype=np.uint8)
        for i,index in enumerate(ID):
            im_name = self.df['Image_id'].loc[index]
            img_path = f"{self.base_path}/{im_name}"
            img =cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            img_i[i,]=img
        return img_i
    
    def __generate_y(self, ID):
        # image name
        img_i = np.empty((self.batch_size,*self.dim,self.n_classes),dtype=np.uint8)
        
        for i,index in enumerate(ID):
            im_name = self.df['Image_id'].loc[index]
            # tran_df
            image_df = self.target_df[self.target_df['Image_id'] == im_name]
            rles = image_df['EncodedPixels'].values
            img =build_masks(rles, input_shape=self.dim)
            img_i[i,]=img
        
        return img_i
    

    
    def __check_input__(self,index):
        X,y=self.__getitem__(index)
        
        # Find list of IDs
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        ID=list_IDs_batch[0]
        X=X[0]
        y=y[0]
        im_name=self.df['Image_id'].loc[ID]
        img= cv2.imread(train_path+im_name)
        image_df = self.target_df[self.target_df['Image_id'] == im_name]
        rles = image_df['EncodedPixels'].values
        masks = build_masks(rles, input_shape=self.dim)
        
        # Plot
        fig,axis = plt.subplots(nrows=1,ncols=1, figsize=(20, 20))
        for i in range(1):
            try:
                img_i=mask_image(X[i].astype(np.uint8),y[i].astype(np.uint8))
                axis[i].imshow(img_i)
            except IndexError: # less the 10 crop 
                print("IndexError : so skip")
                img_i=mask_image(X,y)
                axis.imshow(img_i,cmap='gray')
                
                break
            except TypeError: # only one crop
                img_i=mask_image(X,y)
                print("i")

                axis.imshow(img_i,cmap='gray')
                print("TypeError (axis[i] is fails): so skip")
                break

        img_i=mask_image(img.astype(np.uint8),masks.astype(np.uint8))
        plt.figure(figsize=(10,10))
        plt.imshow(img_i)

In [ ]:
non_missing_train_idx = mask_count[mask_count['mask'] > 0]


In [ ]:

BATCH_SIZE = 10

train_idx, val_idx = train_test_split(
    non_missing_train_idx.index,  # NOTICE DIFFERENCE
    random_state=25, 
    test_size=0.2
)

train_generator = DataGenerator(
    train_idx, 
    df=non_missing_train_idx,
    target_df=train_df,
    batch_size=BATCH_SIZE,
    n_classes=4,
    mode='fit'
)

val_generator = DataGenerator(
    val_idx, 
    df=non_missing_train_idx,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)
# genrator input to model
train_generator.__check_input__(0)

In [ ]:
# metrics
def dice_coef(y_true, y_pred, smooth= 1e-6):
    #y_true_f = K.flatten(y_true)
    #y_pred_f = K.flatten(y_pred)
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred, axis=(1,2))
    return K.mean((2. * intersection + smooth) / (K.sum(y_true, axis=(1,2)) + K.sum(y_pred, axis=(1,2)) + smooth))

# loss function
def dice_loss(y_true, y_pred,smooth = 1e-9):
    intersection = K.sum(y_true * y_pred, axis=(1,2))
    score = (2. * intersection + smooth) / (K.sum(y_true, axis=(1,2)) + K.sum(y_pred, axis=(1,2)) + smooth)
    return K.mean(1-score)

def bce_dice_loss(y_true, y_pred):
    return dice_loss(y_true, y_pred)

In [ ]:
def dice_coef(y_true, y_pred, smooth= 1e-6):
    #y_true_f = K.flatten(y_true)
    #y_pred_f = K.flatten(y_pred)
    y_pred = K.round(y_pred)
    intersection = K.sum(y_true * y_pred,axis=(1,2))
    return K.mean((2. * intersection + smooth) / (K.sum(y_true, axis=(1,2)) + K.sum(y_pred, axis=(1,2)) + smooth))

opt=keras.optimizers.Adam(lr=1e-4)


In [ ]:
#model = sm.Unet(BACKBONE,input_shape=(256, 1600,3),classes=4, activation='sigmoid')
#model.compile(optimizer=opt,loss = 'binary_crossentropy',metrics=[dice_coef])

In [ ]:
es=EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    mode='min'
)
RR = ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = 2, 
        min_lr=1e-6, 
        verbose=1, 
        mode='min')

checkpoint = ModelCheckpoint(
    '1600_FULL.h5', 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=False,
    mode='min',   
)
'''
history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint,es,RR],
    use_multiprocessing=False,
    verbose=1,
    workers=1,
    epochs=20)
'''

In [ ]:
model_seg = load_model('/kaggle/input/seg-model/seg_model .h5')
#model_seg.load_weights('/kaggle/input/16000-model/1600.h5')
model_seg.load_weights('/kaggle/input/16000-model/1600.h5')

In [ ]:
np.sum(pred_class,axis=0)

In [ ]:
sub_df = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')
sub_df['Image_id'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['Image_id'].unique(), columns=['Image_id'])
test_imgs=test_imgs

filtered_mask = sub_df['Image_id'].isin(test_imgs["Image_id"].values)
filtered_sub_df = sub_df[filtered_mask].copy()
null_sub_df = sub_df[~filtered_mask].copy()
null_sub_df['EncodedPixels'] = null_sub_df['EncodedPixels'].apply(
    lambda x: ' ')

filtered_sub_df.reset_index(drop=True, inplace=True)
test_imgs.reset_index(drop=True, inplace=True)

print(filtered_sub_df.shape)
print(null_sub_df.shape)

filtered_sub_df.head()


In [ ]:
all_class=pred_class.flatten()
all_class
filtered_sub_df['EncodedPixels']=all_class

In [ ]:
def sliding_window(img, sz=256):
    total=7
    x=0
    img=img[0]
    new_img = np.empty((total,256,256,3),dtype=np.uint8)
    for count in range(total-1):
        new_img[count,]=img[:, x:x+sz, :]
        x=x+sz
    new_img[-1,]=img[:, -256:, :]
    
    return new_img

In [ ]:
# batch size is fixed and 1 can include  patch size
class DataGenerator2(keras.utils.Sequence):
    'Generates data for Keras'
    
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path=train_path,
                 batch_size=1, dim=(256, 1600), n_channels=3,
                 n_classes=4, random_state=1994, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) // self.batch_size))
    
    
    def __getitem__(self, index):
            'Generate one batch of data'
            # Generate indexes of the batch
            indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
            # Find list of IDs
            list_IDs_batch = [self.list_IDs[k] for k in indexes]


            X = self.__generate_X(list_IDs_batch)

            if self.mode == 'fit':
                y = self.__generate_y(list_IDs_batch)
                return crop_rand_pattern(X, y)

            elif self.mode == 'predict':
                return sliding_window(X)
            
            elif self.mode=='val':
                y = self.__generate_y(list_IDs_batch)
                
                return sliding_window_predict(X, y)
            else:
                raise AttributeError('The mode parameter should be set to "fit" or "predict".')
            
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, ID):
        'Generates data containing batch_size samples'
        # Initialization        
        # Generate data
        img_i = np.empty((self.batch_size,*self.dim,self.n_channels),dtype=np.uint8)
        for i,index in enumerate(ID):
            im_name = self.df['Image_id'].loc[index]
            img_path = f"{self.base_path}/{im_name}"
            img =cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
            img_i[i,]=img
        return img_i
    
    def __generate_y(self, ID):
        # image name
        img_i = np.empty((self.batch_size,*self.dim,self.n_classes),dtype=np.uint8)
        
        for i,index in enumerate(ID):
            im_name = self.df['Image_id'].loc[index]
            # tran_df
            image_df = self.target_df[self.target_df['Image_id'] == im_name]
            rles = image_df['EncodedPixels'].values
            img =build_masks(rles, input_shape=self.dim)
            img_i[i,]=img
        
        return img_i
    

    
    def __check_input__(self,index):
        X,y=self.__getitem__(index)
        
        # Find list of IDs
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        ID=list_IDs_batch
        X=X
        y=y
        for i,ID in enumerate(list_IDs_batch):
            im_name=self.df['Image_id'].loc[ID]
            img= cv2.imread(train_path+im_name)
            image_df = self.target_df[self.target_df['Image_id'] == im_name]
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            img_i=mask_image(img.astype(np.uint8),masks.astype(np.uint8))
            
            plt.figure(figsize=(10,10))
            plt.imshow(img_i)
            plt.show()
            
        
        # Plot
        col=y.shape[0]
        fig,axis = plt.subplots(nrows=1,ncols=col, figsize=(20, 20))
        for i in range(col):
            try:
                img_i=mask_image(X[i].astype(np.uint8),y[i].astype(np.uint8))
                axis[i].imshow(img_i)
            except IndexError: # less the 10 crop 
                print("IndexError : so skip")
                img_i=mask_image(X[i].astype(np.uint8),y[i].astype(np.uint8))
                axis.imshow(img_i,cmap='gray')
                
                break
            except TypeError: # only one crop
                img_i=mask_image(X[i].astype(np.uint8),y[i].astype(np.uint8))
                print("i")

                axis.imshow(img_i,cmap='gray')
                print("TypeError (axis[i] is fails): so skip")
                break

In [ ]:
model_seg_2 = load_model('/kaggle/input/seg-model-256-256/seg_model.h5')
model_seg_2.load_weights('/kaggle/input/seg-weights-898/bce_dice_loss_bce_from_8963_exp_4.h5')

In [ ]:
from keras.models import load_model

test_df = []

for i in range(0, test_imgs.shape[0], 300):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 300))
    )
    
    test_generator = DataGenerator(
        batch_idx,
        df=test_imgs,
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        target_df=test_imgs,
        batch_size=1,
        n_classes=4
    )

    batch_pred_masks = model_seg.predict_generator(
        test_generator, 
        workers=1,
        verbose=1,
        use_multiprocessing=False
    )

    
    for j, b in tqdm(enumerate(batch_idx)):
        filename = test_imgs['Image_id'].iloc[b]
        image_df = filtered_sub_df[filtered_sub_df['Image_id'] == filename].copy()
        
        pred_masks=batch_pred_masks[j,].round().astype(int)
        value = np.sum(pred_masks,axis=(1,0))
        # defect class by seg vs classifier True or flase
        c1=np.array(value*image_df['EncodedPixels'])
        
        pred_masks=batch_pred_masks[j,]
        pred_masks[np.where(pred_masks>0.6)]=1
        pred_masks[np.where(pred_masks<=0.6)]=0
        pred_rles = build_rles(pred_masks)
        
        pred_rles=np.array(pred_rles)
        # if False
        pred_rles[c1==0]=''

        image_df['EncodedPixels'] = pred_rles
        
        test_df.append(image_df)
        
    gc.collect()

In [ ]:
test_df = pd.concat(test_df)
print(test_df.shape)
test_df.head()

In [ ]:
test_df[['ImageId_ClassId', 'EncodedPixels']].to_csv('submission.csv', index=False)

In [ ]:
c1=np.sum(~(test_df.loc[np.arange(1801)*4]['EncodedPixels']=='').values)
c2=np.sum(~(test_df.loc[np.arange(1801)*4+1]['EncodedPixels']=='').values)
c3=np.sum(~(test_df.loc[np.arange(1801)*4+2]['EncodedPixels']=='').values)
c4=np.sum(~(test_df.loc[np.arange(1801)*4+3]['EncodedPixels']=='').values)
print(" class1 ={} \n class2 ={} \n class3 ={} \n class4 ={}".format(c1,c2,c3,c4))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['dice_coef', 'val_dice_coef']].plot()
history_df

In [ ]:
a=np.array([0,2])
a[np.where(a>0.7)]=10
a